In [ ]:
import requests
import pandas as pd
from sklearn.preprocessing import StandardScaler

# API URLs
API_URLS = {
    'viewed': 'https://api.socialverseapp.com/posts/view?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if',
    'liked': 'https://api.socialverseapp.com/posts/like?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if',
    'inspired': 'https://api.socialverseapp.com/posts/inspire?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if',
    'rated': 'https://api.socialverseapp.com/posts/rating?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if',
    'posts': 'https://api.socialverseapp.com/posts/rating?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if',
}

# Authorization token
HEADERS = {
    "Flic-Token": "flic_6e2d8d25dc29a4ddd382c2383a903cf4a688d1a117f6eb43b35a1e7fadbb84b8"
}

def fetch_data(url):
    response = requests.get(url, headers=HEADERS)
    return response.json()

# Fetch the data
viewed_posts = fetch_data(API_URLS['viewed'])
liked_posts = fetch_data(API_URLS['liked'])
inspired_posts = fetch_data(API_URLS['inspired'])
rated_posts = fetch_data(API_URLS['rated'])

# Convert to DataFrames for easier processing
viewed_df = pd.DataFrame(viewed_posts['data'])
liked_df = pd.DataFrame(liked_posts['data'])
inspired_df = pd.DataFrame(inspired_posts['data'])
rated_df = pd.DataFrame(rated_posts['data'])

# Preprocessing: Handling missing values and normalization
def preprocess_data(df):
    df.fillna(0, inplace=True)
    scaler = StandardScaler()
    return pd.DataFrame(scaler.fit_transform(df.select_dtypes(include=['float64', 'int64'])), columns=df.select_dtypes(include=['float64', 'int64']).columns)

# Normalize data
viewed_df = preprocess_data(viewed_df)
liked_df = preprocess_data(liked_df)
inspired_df = preprocess_data(inspired_df)
rated_df = preprocess_data(rated_df)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def content_based_recommendation(user_data, video_metadata):
    return cosine_similarity(user_data, video_metadata)

def collaborative_filtering(user_data, all_users_data):
    return cosine_similarity(user_data, all_users_data)

# Example of combining both
def hybrid_recommendation(user_data, video_metadata, all_users_data):
    content_score = content_based_recommendation(user_data, video_metadata)
    collaborative_score = collaborative_filtering(user_data, all_users_data)
    return (content_score + collaborative_score) / 2


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/feed', methods=['GET'])
def recommend_feed():
    username = request.args.get('username')
    category_id = request.args.get('category_id', default=None)
    mood = request.args.get('mood', default=None)

    # Fetch user data based on username
    user_data = fetch_user_data(username)

    # Fetch category data if category is provided
    if category_id:
        video_metadata = fetch_video_metadata(category_id)
    else:
        video_metadata = fetch_all_video_metadata()

    # Handle cold start: If no user history or mood provided, recommend based on popular videos
    if not user_data:
        recommendations = recommend_popular_videos(mood)
    else:
        recommendations = hybrid_recommendation(user_data, video_metadata, all_users_data)

    return jsonify({'recommended_posts': recommendations[:10]})

def fetch_user_data(username):
    # Placeholder: Fetch user data (interaction history, preferences, etc.)
    return {}

def fetch_video_metadata(category_id):
    # Placeholder: Fetch videos by category
    return {}

def fetch_all_video_metadata():
    # Placeholder: Fetch all available videos
    return {}

def recommend_popular_videos(mood):
    # Placeholder: Recommend based on mood or popular videos
    return []


In [ ]:
if __name__ == '__main__':
    app.run(debug=True)
